In [2]:
import os
import glob
import shutil
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools import mask as cocomask
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from collections import defaultdict
import torch
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.build_sam import build_sam2_video_predictor

In [3]:
# SAM2 model setup
checkpoint = r"C:\Users\dell\Desktop\Assignment\sam2_hiera_tiny.pt"
model_cfg = r"C:\Users\dell\Desktop\Assignment\sam2\configs\sam2\sam2_hiera_t.yaml"


# Initialize SAM2 components
predictor_prompt = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint, device='cpu'))
sam2 = build_sam2(model_cfg, checkpoint, device='cpu', apply_postprocessing=False)
mask_generator = SAM2AutomaticMaskGenerator(sam2)
predictor_vid = build_sam2_video_predictor(model_cfg, checkpoint, device='cpu')

In [4]:
# Temporary directory for processing
tempfolder = "./tempdir"

def create_if_not_exists(dirname):
    if not os.path.exists(dirname):
        os.mkdir(dirname)

def cleardir(tempfolder):
    filepaths = glob.glob(tempfolder+"/*")
    for filepath in filepaths:
        os.unlink(filepath)

In [5]:
def show_mask(mask, ax, obj_id=None, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        cmap = plt.get_cmap("tab10")
        cmap_idx = 0 if obj_id is None else obj_id
        color = np.array([*cmap(cmap_idx)[:3], 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

In [6]:
def track_item_boxes(imgpath1, imgpath2, img1boxclasslist, visualize=True):
    """Track objects between two images using SAM2"""
    create_if_not_exists(tempfolder)
    cleardir(tempfolder)
    shutil.copy(imgpath1, os.path.join(tempfolder, "00000.jpg"))
    shutil.copy(imgpath2, os.path.join(tempfolder, "00001.jpg"))
    
    inference_state = predictor_vid.init_state(video_path="./tempdir")
    predictor_vid.reset_state(inference_state)
    ann_frame_idx = 0
    
    for img1boxclass in img1boxclasslist:
        ([xmin, xmax, ymin, ymax], objectnumint) = img1boxclass
        box = np.array([xmin, ymin, xmax, ymax], dtype=np.float32)
        _, out_obj_ids, out_mask_logits = predictor_vid.add_new_points_or_box(
            inference_state=inference_state,
            frame_idx=ann_frame_idx,
            obj_id=objectnumint,
            box=box,
        )
    
    video_segments = {}
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor_vid.propagate_in_video(inference_state):
        print(f"\nDebug: Frame {out_frame_idx} has {len(out_obj_ids)} objects")  # Debug print
        print(f"Object IDs: {out_obj_ids}")  # Debug print
        
        video_segments[out_frame_idx] = {
            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
            for i, out_obj_id in enumerate(out_obj_ids)
        }
    
    if visualize:
        fig, ax = plt.subplots()
        plt.title(f"original image object ::")
        ax.imshow(Image.open(os.path.join(tempfolder, "00000.jpg")))
        rect = patches.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, 
                               linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.show()
        
        out_frame_idx = 1
        plt.figure(figsize=(6, 4))
        plt.title(f"detected object in test image ::")
        plt.imshow(Image.open(os.path.join(tempfolder, "00001.jpg")))
        for out_obj_id, out_mask in video_segments[out_frame_idx].items():
            show_mask(out_mask, plt.gca(), obj_id=out_obj_id)
        plt.show()
    
    return video_segments

In [7]:
def extract_category_from_filename(filename):
    """Extract category name from filename"""
    return '_'.join(os.path.basename(filename).split('_')[:-1])

def group_files_by_category(data_dir):
    """Group image and mask files by their product category"""
    image_files = glob.glob(os.path.join(data_dir, "*.jpg"))
    mask_files = glob.glob(os.path.join(data_dir, "*_gt.png"))
    
    category_dict = defaultdict(lambda: {'images': [], 'masks': []})
    
    for img in image_files:
        category = extract_category_from_filename(img)
        category_dict[category]['images'].append(img)
    
    for mask in mask_files:
        base_name = '_'.join(os.path.basename(mask).split('_')[:-2])
        category = extract_category_from_filename(base_name + '.jpg')
        
        matching_images = [img for img in category_dict[category]['images'] 
                         if base_name in img]
        
        for img in matching_images:
            category_dict[category]['masks'].append((img, mask))
    
    return category_dict

In [8]:
def process_img_png_mask(img_path, mask_path, visualize=False):
    """Extract bounding box coordinates from mask"""
    try:
        img = Image.open(img_path)
        mask = Image.open(mask_path)
        mask_array = np.array(mask)
        
        if mask_array.size == 0:
            print(f"Warning: Empty mask in {mask_path}")
            return 0, 0, 0, 0
            
        rows = np.any(mask_array, axis=1)
        cols = np.any(mask_array, axis=0)
        
        ymin, ymax = np.where(rows)[0][[0, -1]]
        xmin, xmax = np.where(cols)[0][[0, -1]]
        
        if visualize:
            fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].imshow(img)
            ax[0].set_title("Original Image")
            ax[1].imshow(mask_array)
            rect = plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, 
                               linewidth=1, edgecolor='r', facecolor='none')
            ax[1].add_patch(rect)
            ax[1].set_title("Mask with Bounding Box")
            plt.show()
        
        return xmin, xmax, ymin, ymax
        
    except Exception as e:
        print(f"Error processing {img_path}: {str(e)}")
        return 0, 0, 0, 0

In [9]:
def mask_to_bbox(mask):
    """Convert binary mask to COCO-style bounding box [x,y,width,height]"""
    # Handle 3D mask by taking first channel if needed
    if len(mask.shape) == 3:
        mask = mask[0] if mask.shape[0] == 1 else mask
    
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    
    if not np.any(rows) or not np.any(cols):
        return [0, 0, 0, 0]  # Return empty bbox if no True values
    
    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    
    return [int(xmin), int(ymin), int(xmax-xmin), int(ymax-ymin)]

In [10]:
def evaluate_product_category(category_name, image_mask_pairs):
    """Evaluate SAM2 performance on one product category"""
    print(f"\nEvaluating {category_name}...")
    
    if not image_mask_pairs or len(image_mask_pairs) < 2:
        print("Error: Need at least 2 image-mask pairs for evaluation")
        return None
    
    # Use first image-mask pair as reference
    first_img, first_mask = image_mask_pairs[0]
    print(f"Using reference image: {os.path.basename(first_img)}")
    print(f"Using reference mask: {os.path.basename(first_mask)}")
    
    xmin, xmax, ymin, ymax = process_img_png_mask(first_img, first_mask)
    print(f"Reference bounding box: x=[{xmin},{xmax}], y=[{ymin},{ymax}]")
    
    coco_gt = {"images": [], "annotations": [], "categories": [{"id": 1, "name": "product"}]}
    coco_dt = []
    
    for i, (img_path, mask_path) in enumerate(image_mask_pairs[1:]):
        print(f"\nProcessing image {i+1}/{len(image_mask_pairs)-1}: {os.path.basename(img_path)}")
        
        try:
            # Process ground truth
            gt_mask = np.array(Image.open(mask_path))
            gt_bbox = mask_to_bbox(gt_mask)
            print(f"Ground truth bbox: {gt_bbox}")
            
            # Add to COCO ground truth
            image_id = i+1
            coco_gt["images"].append({"id": image_id, "file_name": img_path})
            
            rle = cocomask.encode(np.asfortranarray(gt_mask))
            rle['counts'] = rle['counts'].decode('ascii')
            
            coco_gt["annotations"].append({
                "id": image_id,
                "image_id": image_id,
                "category_id": 1,
                "bbox": gt_bbox,
                "area": int(gt_bbox[2] * gt_bbox[3]),
                "iscrowd": 0,
                "segmentation": rle
            })
            
            # Get SAM2 prediction
            video_segments = track_item_boxes(first_img, img_path, [([xmin, xmax, ymin, ymax], 1)], False)
            
            if not video_segments or 1 not in video_segments:
                print("Warning: No valid segments found in frame 1")
                continue
                
            # Get the prediction mask
            pred_mask = video_segments[1][1]  # Frame 1, Object ID 1
            
            # Convert mask to correct format if needed
            if len(pred_mask.shape) == 3:
                pred_mask = pred_mask[0] if pred_mask.shape[0] == 1 else pred_mask
            
            pred_bbox = mask_to_bbox(pred_mask)
            print(f"Predicted bbox: {pred_bbox}")
            
            # Skip if prediction is empty
            if pred_bbox == [0, 0, 0, 0]:
                print("Warning: Empty prediction - skipping")
                continue
                
            # Convert prediction to COCO format
            pred_rle = cocomask.encode(np.asfortranarray(pred_mask.astype(np.uint8)))
            pred_rle['counts'] = pred_rle['counts'].decode('ascii')
            
            coco_dt.append({
                "image_id": image_id,
                "category_id": 1,
                "bbox": pred_bbox,
                "score": 1.0,
                "segmentation": pred_rle
            })
            
        except Exception as e:
            print(f"Error processing {img_path}: {str(e)}")
            continue
    
    if not coco_dt:
        print("Error: No valid predictions generated!")
        return None
    
    try:
        coco_gt_obj = COCO()
        coco_gt_obj.dataset = coco_gt
        coco_gt_obj.createIndex()
        
        coco_dt_obj = coco_gt_obj.loadRes(coco_dt)
        coco_eval = COCOeval(coco_gt_obj, coco_dt_obj, 'bbox')
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()
        
        return coco_eval.stats
        
    except Exception as e:
        print(f"Error in COCO evaluation: {str(e)}")
        return None

In [11]:
def main():
    data_dir = r"C:\Users\dell\Desktop\Assignment\CMU10_3D\data_2D"
    
    # Group files by category
    category_dict = group_files_by_category(data_dir)
    
    # Evaluate each category
    results = {}
    for category, files in category_dict.items():
        # Create image-mask pairs
        image_mask_pairs = []
        for img in files['images']:
            base_name = os.path.splitext(os.path.basename(img))[0]
            masks = [mask for (img_path, mask) in files['masks'] 
                    if base_name in os.path.basename(mask)]
            
            if masks:
                image_mask_pairs.append((img, masks[0]))  # Using first mask
        
        if len(image_mask_pairs) < 2:
            print(f"Skipping {category} - needs at least 2 images with masks")
            continue
            
        stats = evaluate_product_category(category, image_mask_pairs)
        
        if stats is not None:
            results[category] = {
                "AP": stats[0],  # AP @ IoU=0.50:0.95
                "AP50": stats[1],  # AP @ IoU=0.50
                "AP75": stats[2],  # AP @ IoU=0.75
                "AR": stats[8]     # AR @ maxDets=100
            }
    
    # Print results
    print("\n=== Final Results ===")
    for product, metrics in results.items():
        print(f"\n{product}:")
        print(f"  AP: {metrics['AP']:.3f}")
        print(f"  AP50: {metrics['AP50']:.3f}")
        print(f"  AP75: {metrics['AP75']:.3f}")
        print(f"  AR: {metrics['AR']:.3f}")

if __name__ == "__main__":
    main()


Evaluating can_chowder...
Using reference image: can_chowder_000001.jpg
Using reference mask: can_chowder_000001_1_gt.png
Reference bounding box: x=[371,464], y=[150,290]

Processing image 1/49: can_chowder_000002.jpg
Ground truth bbox: [249, 148, 87, 130]


frame loading (JPEG): 100%|██████████| 2/2 [00:00<00:00,  9.73it/s]
c:\Users\dell\Desktop\Assignment\sam2\sam2_video_predictor.py:786: UserWarning: cannot import name '_C' from 'sam2' (c:\Users\dell\Desktop\Assignment\sam2\__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video:   0%|          | 0/2 [00:00<?, ?it/s]


Debug: Frame 0 has 1 objects
Object IDs: [1]


propagate in video: 100%|██████████| 2/2 [00:05<00:00,  2.66s/it]



Debug: Frame 1 has 1 objects
Object IDs: [1]
Predicted bbox: [249, 150, 82, 128]

Processing image 2/49: can_chowder_000003.jpg
Ground truth bbox: [398, 92, 105, 146]


...


Debug: Frame 1 has 1 objects
Object IDs: [1]
Predicted bbox: [0, 0, 0, 0]

Processing image 49/49: rice_tuscan_000050.jpg
Ground truth bbox: [347, 234, 119, 205]


propagate in video:   0%|          | 0/2 [00:00<?, ?it/s]


Debug: Frame 0 has 1 objects
Object IDs: [1]


propagate in video: 100%|██████████| 2/2 [00:04<00:00,  2.46s/it]


Debug: Frame 1 has 1 objects
Object IDs: [1]
Predicted bbox: [6, 137, 562, 339]
creating index...
index created!
Error in COCO evaluation: 'info'

=== Final Results ===


In [ ]:
import os
import glob
import shutil
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools import mask as cocomask
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from collections import defaultdict
import torch
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.build_sam import build_sam2_video_predictor


# SAM2 model setup
checkpoint = r"C:\Users\dell\Desktop\Assignment\sam2_hiera_tiny.pt"
model_cfg = r"C:\Users\dell\Desktop\Assignment\sam2\configs\sam2\sam2_hiera_t.yaml"


# Initialize SAM2 components
predictor_prompt = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint, device='cpu'))
sam2 = build_sam2(model_cfg, checkpoint, device='cpu', apply_postprocessing=False)
mask_generator = SAM2AutomaticMaskGenerator(sam2)
predictor_vid = build_sam2_video_predictor(model_cfg, checkpoint, device='cpu')

# Temporary directory for processing
tempfolder = "./tempdir"

def create_if_not_exists(dirname):
    if not os.path.exists(dirname):
        os.mkdir(dirname)

def cleardir(tempfolder):
    filepaths = glob.glob(tempfolder+"/*")
    for filepath in filepaths:
        os.unlink(filepath)

def show_mask(mask, ax, obj_id=None, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        cmap = plt.get_cmap("tab10")
        cmap_idx = 0 if obj_id is None else obj_id
        color = np.array([*cmap(cmap_idx)[:3], 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def track_item_boxes(imgpath1, imgpath2, img1boxclasslist, visualize=True):
    """Track objects between two images using SAM2"""
    create_if_not_exists(tempfolder)
    cleardir(tempfolder)
    shutil.copy(imgpath1, os.path.join(tempfolder, "00000.jpg"))
    shutil.copy(imgpath2, os.path.join(tempfolder, "00001.jpg"))
    
    inference_state = predictor_vid.init_state(video_path="./tempdir")
    predictor_vid.reset_state(inference_state)
    ann_frame_idx = 0
    
    for img1boxclass in img1boxclasslist:
        ([xmin, xmax, ymin, ymax], objectnumint) = img1boxclass
        box = np.array([xmin, ymin, xmax, ymax], dtype=np.float32)
        _, out_obj_ids, out_mask_logits = predictor_vid.add_new_points_or_box(
            inference_state=inference_state,
            frame_idx=ann_frame_idx,
            obj_id=objectnumint,
            box=box,
        )
    
    video_segments = {}
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor_vid.propagate_in_video(inference_state):
        print(f"\nDebug: Frame {out_frame_idx} has {len(out_obj_ids)} objects")  # Debug print
        print(f"Object IDs: {out_obj_ids}")  # Debug print
        
        video_segments[out_frame_idx] = {
            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
            for i, out_obj_id in enumerate(out_obj_ids)
        }
    
    if visualize:
        fig, ax = plt.subplots()
        plt.title(f"original image object ::")
        ax.imshow(Image.open(os.path.join(tempfolder, "00000.jpg")))
        rect = patches.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, 
                               linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.show()
        
        out_frame_idx = 1
        plt.figure(figsize=(6, 4))
        plt.title(f"detected object in test image ::")
        plt.imshow(Image.open(os.path.join(tempfolder, "00001.jpg")))
        for out_obj_id, out_mask in video_segments[out_frame_idx].items():
            show_mask(out_mask, plt.gca(), obj_id=out_obj_id)
        plt.show()
    
    return video_segments

def extract_category_from_filename(filename):
    """Extract category name from filename"""
    return '_'.join(os.path.basename(filename).split('_')[:-1])

def group_files_by_category(data_dir):
    """Group image and mask files by their product category"""
    image_files = glob.glob(os.path.join(data_dir, "*.jpg"))
    mask_files = glob.glob(os.path.join(data_dir, "*_gt.png"))
    
    category_dict = defaultdict(lambda: {'images': [], 'masks': []})
    
    for img in image_files:
        category = extract_category_from_filename(img)
        category_dict[category]['images'].append(img)
    
    for mask in mask_files:
        base_name = '_'.join(os.path.basename(mask).split('_')[:-2])
        category = extract_category_from_filename(base_name + '.jpg')
        
        matching_images = [img for img in category_dict[category]['images'] 
                         if base_name in img]
        
        for img in matching_images:
            category_dict[category]['masks'].append((img, mask))
    
    return category_dict

def process_img_png_mask(img_path, mask_path, visualize=False):
    """Extract bounding box coordinates from mask"""
    try:
        img = Image.open(img_path)
        mask = Image.open(mask_path)
        mask_array = np.array(mask)
        
        if mask_array.size == 0:
            print(f"Warning: Empty mask in {mask_path}")
            return 0, 0, 0, 0
            
        rows = np.any(mask_array, axis=1)
        cols = np.any(mask_array, axis=0)
        
        ymin, ymax = np.where(rows)[0][[0, -1]]
        xmin, xmax = np.where(cols)[0][[0, -1]]
        
        if visualize:
            fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].imshow(img)
            ax[0].set_title("Original Image")
            ax[1].imshow(mask_array)
            rect = plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, 
                               linewidth=1, edgecolor='r', facecolor='none')
            ax[1].add_patch(rect)
            ax[1].set_title("Mask with Bounding Box")
            plt.show()
        
        return xmin, xmax, ymin, ymax
        
    except Exception as e:
        print(f"Error processing {img_path}: {str(e)}")
        return 0, 0, 0, 0

def mask_to_bbox(mask):
    """Convert binary mask to COCO-style bounding box [x,y,width,height]"""
    # Handle 3D mask by taking first channel if needed
    if len(mask.shape) == 3:
        mask = mask[0] if mask.shape[0] == 1 else mask
    
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    
    if not np.any(rows) or not np.any(cols):
        return [0, 0, 0, 0]  # Return empty bbox if no True values
    
    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    
    return [int(xmin), int(ymin), int(xmax-xmin), int(ymax-ymin)]

def evaluate_product_category(category_name, image_mask_pairs):
    """Evaluate SAM2 performance on one product category"""
    print(f"\nEvaluating {category_name}...")
    
    if not image_mask_pairs or len(image_mask_pairs) < 2:
        print("Error: Need at least 2 image-mask pairs for evaluation")
        return None
    
    # Use first image-mask pair as reference
    first_img, first_mask = image_mask_pairs[0]
    print(f"Using reference image: {os.path.basename(first_img)}")
    print(f"Using reference mask: {os.path.basename(first_mask)}")
    
    xmin, xmax, ymin, ymax = process_img_png_mask(first_img, first_mask)
    print(f"Reference bounding box: x=[{xmin},{xmax}], y=[{ymin},{ymax}]")
    
    coco_gt = {"images": [], "annotations": [], "categories": [{"id": 1, "name": "product"}]}
    coco_dt = []
    
    for i, (img_path, mask_path) in enumerate(image_mask_pairs[1:]):
        print(f"\nProcessing image {i+1}/{len(image_mask_pairs)-1}: {os.path.basename(img_path)}")
        
        try:
            # Process ground truth
            gt_mask = np.array(Image.open(mask_path))
            gt_bbox = mask_to_bbox(gt_mask)
            print(f"Ground truth bbox: {gt_bbox}")
            
            # Add to COCO ground truth
            image_id = i+1
            coco_gt["images"].append({"id": image_id, "file_name": img_path})
            
            rle = cocomask.encode(np.asfortranarray(gt_mask))
            rle['counts'] = rle['counts'].decode('ascii')
            
            coco_gt["annotations"].append({
                "id": image_id,
                "image_id": image_id,
                "category_id": 1,
                "bbox": gt_bbox,
                "area": int(gt_bbox[2] * gt_bbox[3]),
                "iscrowd": 0,
                "segmentation": rle
            })
            
            # Get SAM2 prediction
            video_segments = track_item_boxes(first_img, img_path, [([xmin, xmax, ymin, ymax], 1)], False)
            
            if not video_segments or 1 not in video_segments:
                print("Warning: No valid segments found in frame 1")
                continue
                
            # Get the prediction mask
            pred_mask = video_segments[1][1]  # Frame 1, Object ID 1
            
            # Convert mask to correct format if needed
            if len(pred_mask.shape) == 3:
                pred_mask = pred_mask[0] if pred_mask.shape[0] == 1 else pred_mask
            
            pred_bbox = mask_to_bbox(pred_mask)
            print(f"Predicted bbox: {pred_bbox}")
            
            # Skip if prediction is empty
            if pred_bbox == [0, 0, 0, 0]:
                print("Warning: Empty prediction - skipping")
                continue
                
            # Convert prediction to COCO format
            pred_rle = cocomask.encode(np.asfortranarray(pred_mask.astype(np.uint8)))
            pred_rle['counts'] = pred_rle['counts'].decode('ascii')
            
            coco_dt.append({
                "image_id": image_id,
                "category_id": 1,
                "bbox": pred_bbox,
                "score": 1.0,
                "segmentation": pred_rle
            })
            
        except Exception as e:
            print(f"Error processing {img_path}: {str(e)}")
            continue
    
    if not coco_dt:
        print("Error: No valid predictions generated!")
        return None
    
    try:
        coco_gt_obj = COCO()
        coco_gt_obj.dataset = coco_gt
        coco_gt_obj.createIndex()
        
        coco_dt_obj = coco_gt_obj.loadRes(coco_dt)
        coco_eval = COCOeval(coco_gt_obj, coco_dt_obj, 'bbox')
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()
        
        return coco_eval.stats
        
    except Exception as e:
        print(f"Error in COCO evaluation: {str(e)}")
        return None
    
def main():
    data_dir = r"C:\Users\dell\Desktop\Assignment\CMU10_3D\data_2D"
    
    # Group files by category
    category_dict = group_files_by_category(data_dir)
    
    # Evaluate each category
    results = {}
    for category, files in category_dict.items():
        # Create image-mask pairs
        image_mask_pairs = []
        for img in files['images']:
            base_name = os.path.splitext(os.path.basename(img))[0]
            masks = [mask for (img_path, mask) in files['masks'] 
                    if base_name in os.path.basename(mask)]
            
            if masks:
                image_mask_pairs.append((img, masks[0]))  # Using first mask
        
        if len(image_mask_pairs) < 2:
            print(f"Skipping {category} - needs at least 2 images with masks")
            continue
            
        stats = evaluate_product_category(category, image_mask_pairs)
        
        if stats is not None:
            results[category] = {
                "AP": stats[0],  # AP @ IoU=0.50:0.95
                "AP50": stats[1],  # AP @ IoU=0.50
                "AP75": stats[2],  # AP @ IoU=0.75
                "AR": stats[8]     # AR @ maxDets=100
            }
    
    # Print results
    print("\n=== Final Results ===")
    for product, metrics in results.items():
        print(f"\n{product}:")
        print(f"  AP: {metrics['AP']:.3f}")
        print(f"  AP50: {metrics['AP50']:.3f}")
        print(f"  AP75: {metrics['AP75']:.3f}")
        print(f"  AR: {metrics['AR']:.3f}")

if __name__ == "__main__":
    main()

In [12]:
import os
import glob
import shutil
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools import mask as cocomask
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from collections import defaultdict
import torch
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.build_sam import build_sam2_video_predictor
from IPython.display import clear_output
import sys
import warnings

# Suppress warnings and limit output
warnings.filterwarnings('ignore')

# SAM2 model setup
checkpoint = r"C:\Users\dell\Desktop\Assignment\sam2_hiera_tiny.pt"
model_cfg = r"C:\Users\dell\Desktop\Assignment\sam2\configs\sam2\sam2_hiera_t.yaml"

# Initialize SAM2 components
predictor_prompt = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint, device='cpu'))
sam2 = build_sam2(model_cfg, checkpoint, device='cpu', apply_postprocessing=False)
mask_generator = SAM2AutomaticMaskGenerator(sam2)
predictor_vid = build_sam2_video_predictor(model_cfg, checkpoint, device='cpu')

# Temporary directory for processing
tempfolder = "./tempdir"

def create_if_not_exists(dirname):
    if not os.path.exists(dirname):
        os.mkdir(dirname)

def cleardir(tempfolder):
    filepaths = glob.glob(tempfolder+"/*")
    for filepath in filepaths:
        os.unlink(filepath)

def show_mask(mask, ax, obj_id=None, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        cmap = plt.get_cmap("tab10")
        cmap_idx = 0 if obj_id is None else obj_id
        color = np.array([*cmap(cmap_idx)[:3], 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def track_item_boxes(imgpath1, imgpath2, img1boxclasslist, visualize=True):
    """Enhanced tracking with padding and better error handling"""
    try:
        create_if_not_exists(tempfolder)
        cleardir(tempfolder)
        
        # Load and verify images
        img1 = Image.open(imgpath1)
        img2 = Image.open(imgpath2)
        if img1.size != img2.size:
            img2 = img2.resize(img1.size)
        
        img1.save(os.path.join(tempfolder, "00000.jpg"))
        img2.save(os.path.join(tempfolder, "00001.jpg"))
        
        inference_state = predictor_vid.init_state(video_path="./tempdir")
        predictor_vid.reset_state(inference_state)
        
        # Process boxes with padding
        for ([xmin, xmax, ymin, ymax], obj_id) in img1boxclasslist:
            # Add 10% padding to the bounding box
            width, height = img1.size
            pad_x = int(0.1 * (xmax - xmin))
            pad_y = int(0.1 * (ymax - ymin))
            
            xmin = max(0, xmin - pad_x)
            xmax = min(width, xmax + pad_x)
            ymin = max(0, ymin - pad_y)
            ymax = min(height, ymax + pad_y)
            
            box = np.array([xmin, ymin, xmax, ymax], dtype=np.float32)
            _, out_obj_ids, _ = predictor_vid.add_new_points_or_box(
                inference_state=inference_state,
                frame_idx=0,
                obj_id=obj_id,
                box=box,
            )
        
        # Get predictions with threshold adjustment
        video_segments = {}
        for out_frame_idx, out_obj_ids, out_mask_logits in predictor_vid.propagate_in_video(inference_state):
            video_segments[out_frame_idx] = {
                obj_id: (out_mask_logits[i] > 0.5).cpu().numpy().astype(bool)
                for i, obj_id in enumerate(out_obj_ids)
            }
        
        if visualize:
            fig, ax = plt.subplots()
            plt.title(f"Original image object")
            ax.imshow(img1)
            rect = patches.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin,
                                   linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            plt.show()
            
            plt.figure(figsize=(6, 4))
            plt.title(f"Detected object in test image")
            plt.imshow(img2)
            if 1 in video_segments:
                for out_obj_id, out_mask in video_segments[1].items():
                    show_mask(out_mask, plt.gca(), obj_id=out_obj_id)
            plt.show()
        
        return video_segments
        
    except Exception as e:
        print(f"Tracking error: {str(e)}")
        return None

def extract_category_from_filename(filename):
    """Extract category name from filename"""
    return '_'.join(os.path.basename(filename).split('_')[:-1])

def group_files_by_category(data_dir):
    """Group image and mask files by their product category"""
    image_files = glob.glob(os.path.join(data_dir, "*.jpg"))
    mask_files = glob.glob(os.path.join(data_dir, "*_gt.png"))
    
    category_dict = defaultdict(lambda: {'images': [], 'masks': []})
    
    for img in image_files:
        category = extract_category_from_filename(img)
        category_dict[category]['images'].append(img)
    
    for mask in mask_files:
        base_name = '_'.join(os.path.basename(mask).split('_')[:-2])
        category = extract_category_from_filename(base_name + '.jpg')
        
        matching_images = [img for img in category_dict[category]['images'] 
                         if base_name in img]
        
        for img in matching_images:
            category_dict[category]['masks'].append((img, mask))
    
    return category_dict

def process_img_png_mask(img_path, mask_path, visualize=False):
    """Extract bounding box coordinates from mask"""
    try:
        img = Image.open(img_path)
        mask = Image.open(mask_path)
        mask_array = np.array(mask)
        
        if mask_array.size == 0:
            print(f"Warning: Empty mask in {mask_path}")
            return 0, 0, 0, 0
            
        rows = np.any(mask_array, axis=1)
        cols = np.any(mask_array, axis=0)
        
        ymin, ymax = np.where(rows)[0][[0, -1]]
        xmin, xmax = np.where(cols)[0][[0, -1]]
        
        if visualize:
            fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].imshow(img)
            ax[0].set_title("Original Image")
            ax[1].imshow(mask_array)
            rect = plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, 
                               linewidth=1, edgecolor='r', facecolor='none')
            ax[1].add_patch(rect)
            ax[1].set_title("Mask with Bounding Box")
            plt.show()
        
        return xmin, xmax, ymin, ymax
        
    except Exception as e:
        print(f"Error processing {img_path}: {str(e)}")
        return 0, 0, 0, 0

def mask_to_bbox(mask):
    """Improved mask to bbox conversion with better handling"""
    # Handle 3D mask by taking first channel if needed
    if len(mask.shape) == 3:
        mask = mask[0] if mask.shape[0] == 1 else mask
    
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    
    if not np.any(rows) or not np.any(cols):
        return [0, 0, 0, 0]  # Return empty bbox if no True values
    
    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    
    return [int(xmin), int(ymin), int(xmax-xmin), int(ymax-ymin)]

def evaluate_product_category(category_name, image_mask_pairs, max_display=10):
    """Enhanced evaluation with output management"""
    print(f"\nEvaluating {category_name} (showing first {max_display} outputs)...")
    
    if not image_mask_pairs or len(image_mask_pairs) < 2:
        print("Error: Need at least 2 image-mask pairs for evaluation")
        return None
    
    # Use first image-mask pair as reference
    first_img, first_mask = image_mask_pairs[0]
    print(f"Using reference image: {os.path.basename(first_img)}")
    print(f"Using reference mask: {os.path.basename(first_mask)}")
    
    xmin, xmax, ymin, ymax = process_img_png_mask(first_img, first_mask)
    print(f"Reference bounding box: x=[{xmin},{xmax}], y=[{ymin},{ymax}]")
    
    coco_gt = {"images": [], "annotations": [], "categories": [{"id": 1, "name": "product"}]}
    coco_dt = []
    
    for i, (img_path, mask_path) in enumerate(image_mask_pairs[1:]):
        if i >= max_display:  # Only show first N outputs
            clear_output(wait=True)
            print(f"Processing {len(image_mask_pairs)-1} images, showing first {max_display}...")
            break
            
        print(f"\nProcessing image {i+1}/{len(image_mask_pairs)-1}: {os.path.basename(img_path)}")
        
        try:
            # Process ground truth
            gt_mask = np.array(Image.open(mask_path))
            gt_bbox = mask_to_bbox(gt_mask)
            print(f"Ground truth bbox: {gt_bbox}")
            
            # Add to COCO ground truth
            image_id = i+1
            coco_gt["images"].append({"id": image_id, "file_name": img_path})
            
            rle = cocomask.encode(np.asfortranarray(gt_mask))
            rle['counts'] = rle['counts'].decode('ascii')
            
            coco_gt["annotations"].append({
                "id": image_id,
                "image_id": image_id,
                "category_id": 1,
                "bbox": gt_bbox,
                "area": int(gt_bbox[2] * gt_bbox[3]),
                "iscrowd": 0,
                "segmentation": rle
            })
            
            # Get SAM2 prediction
            video_segments = track_item_boxes(first_img, img_path, [([xmin, xmax, ymin, ymax], 1)], False)
            
            if not video_segments or 1 not in video_segments:
                print("Warning: No valid segments found in frame 1")
                continue
                
            # Get the prediction mask
            pred_mask = video_segments[1][1]  # Frame 1, Object ID 1
            
            # Convert mask to correct format if needed
            if len(pred_mask.shape) == 3:
                pred_mask = pred_mask[0] if pred_mask.shape[0] == 1 else pred_mask
            
            pred_bbox = mask_to_bbox(pred_mask)
            print(f"Predicted bbox: {pred_bbox}")
            
            # Skip if prediction is empty
            if pred_bbox == [0, 0, 0, 0]:
                print("Warning: Empty prediction - skipping")
                continue
                
            # Convert prediction to COCO format
            pred_rle = cocomask.encode(np.asfortranarray(pred_mask.astype(np.uint8)))
            pred_rle['counts'] = pred_rle['counts'].decode('ascii')
            
            coco_dt.append({
                "image_id": image_id,
                "category_id": 1,
                "bbox": pred_bbox,
                "score": 1.0,
                "segmentation": pred_rle
            })
            
        except Exception as e:
            print(f"Error processing {img_path}: {str(e)}")
            continue
    
    if not coco_dt:
        print("Error: No valid predictions generated!")
        return None
    
    try:
        coco_gt_obj = COCO()
        coco_gt_obj.dataset = coco_gt
        coco_gt_obj.createIndex()
        
        coco_dt_obj = coco_gt_obj.loadRes(coco_dt)
        coco_eval = COCOeval(coco_gt_obj, coco_dt_obj, 'bbox')
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()
        
        return coco_eval.stats
        
    except Exception as e:
        print(f"Error in COCO evaluation: {str(e)}")
        return None

def main():
    data_dir = r"C:\Users\dell\Desktop\Assignment\CMU10_3D\data_2D"
    
    # Group files by category
    category_dict = group_files_by_category(data_dir)
    
    # Evaluate each category
    results = {}
    for category, files in category_dict.items():
        # Create image-mask pairs
        image_mask_pairs = []
        for img in files['images']:
            base_name = os.path.splitext(os.path.basename(img))[0]
            masks = [mask for (img_path, mask) in files['masks'] 
                    if base_name in os.path.basename(mask)]
            
            if masks:
                image_mask_pairs.append((img, masks[0]))  # Using first mask
        
        if len(image_mask_pairs) < 2:
            print(f"Skipping {category} - needs at least 2 images with masks")
            continue
            
        # Save output to file
        output_file = f"{category}_results.txt"
        with open(output_file, 'w') as f:
            original_stdout = sys.stdout
            sys.stdout = f
            
            stats = evaluate_product_category(category, image_mask_pairs)
            
            sys.stdout = original_stdout
            print(f"Results saved to {output_file}")
        
        if stats is not None:
            results[category] = {
                "AP": stats[0],  # AP @ IoU=0.50:0.95
                "AP50": stats[1],  # AP @ IoU=0.50
                "AP75": stats[2],  # AP @ IoU=0.75
                "AR": stats[8]     # AR @ maxDets=100
            }
    
    # Print final results
    print("\n=== Final Results ===")
    for product, metrics in results.items():
        print(f"\n{product}:")
        print(f"  AP: {metrics['AP']:.3f}")
        print(f"  AP50: {metrics['AP50']:.3f}")
        print(f"  AP75: {metrics['AP75']:.3f}")
        print(f"  AR: {metrics['AR']:.3f}")

if __name__ == "__main__":
    main()

Results saved to rice_tuscan_results.txt

=== Final Results ===


In [13]:
import os
import glob
import shutil
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools import mask as cocomask
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from collections import defaultdict
import torch
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.build_sam import build_sam2_video_predictor
from IPython.display import clear_output
import sys
import warnings
from io import StringIO

# Suppress warnings and limit output
warnings.filterwarnings('ignore')

# SAM2 model setup
checkpoint = r"C:\Users\dell\Desktop\Assignment\sam2_hiera_tiny.pt"
model_cfg = r"C:\Users\dell\Desktop\Assignment\sam2\configs\sam2\sam2_hiera_t.yaml"

# Initialize SAM2 components
predictor_prompt = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint, device='cpu'))
sam2 = build_sam2(model_cfg, checkpoint, device='cpu', apply_postprocessing=False)
mask_generator = SAM2AutomaticMaskGenerator(sam2)
predictor_vid = build_sam2_video_predictor(model_cfg, checkpoint, device='cpu')

# Temporary directory for processing
tempfolder = "./tempdir"

def create_if_not_exists(dirname):
    if not os.path.exists(dirname):
        os.mkdir(dirname)

def cleardir(tempfolder):
    filepaths = glob.glob(tempfolder+"/*")
    for filepath in filepaths:
        os.unlink(filepath)

def show_mask(mask, ax, obj_id=None, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        cmap = plt.get_cmap("tab10")
        cmap_idx = 0 if obj_id is None else obj_id
        color = np.array([*cmap(cmap_idx)[:3], 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def track_item_boxes(imgpath1, imgpath2, img1boxclasslist, visualize=True):
    """Enhanced tracking with padding and better error handling"""
    try:
        create_if_not_exists(tempfolder)
        cleardir(tempfolder)
        
        # Load and verify images
        img1 = Image.open(imgpath1)
        img2 = Image.open(imgpath2)
        if img1.size != img2.size:
            img2 = img2.resize(img1.size)
        
        img1.save(os.path.join(tempfolder, "00000.jpg"))
        img2.save(os.path.join(tempfolder, "00001.jpg"))
        
        inference_state = predictor_vid.init_state(video_path="./tempdir")
        predictor_vid.reset_state(inference_state)
        
        # Process boxes with padding
        for ([xmin, xmax, ymin, ymax], obj_id) in img1boxclasslist:
            # Add 10% padding to the bounding box
            width, height = img1.size
            pad_x = int(0.1 * (xmax - xmin))
            pad_y = int(0.1 * (ymax - ymin))
            
            xmin = max(0, xmin - pad_x)
            xmax = min(width, xmax + pad_x)
            ymin = max(0, ymin - pad_y)
            ymax = min(height, ymax + pad_y)
            
            box = np.array([xmin, ymin, xmax, ymax], dtype=np.float32)
            _, out_obj_ids, _ = predictor_vid.add_new_points_or_box(
                inference_state=inference_state,
                frame_idx=0,
                obj_id=obj_id,
                box=box,
            )
        
        # Get predictions with threshold adjustment
        video_segments = {}
        for out_frame_idx, out_obj_ids, out_mask_logits in predictor_vid.propagate_in_video(inference_state):
            video_segments[out_frame_idx] = {
                obj_id: (out_mask_logits[i] > 0.5).cpu().numpy().astype(bool)
                for i, obj_id in enumerate(out_obj_ids)
            }
        
        if visualize:
            fig, ax = plt.subplots()
            plt.title(f"Original image object")
            ax.imshow(img1)
            rect = patches.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin,
                                   linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            plt.show()
            
            plt.figure(figsize=(6, 4))
            plt.title(f"Detected object in test image")
            plt.imshow(img2)
            if 1 in video_segments:
                for out_obj_id, out_mask in video_segments[1].items():
                    show_mask(out_mask, plt.gca(), obj_id=out_obj_id)
            plt.show()
        
        return video_segments
        
    except Exception as e:
        print(f"Tracking error: {str(e)}")
        return None

def extract_category_from_filename(filename):
    """Extract category name from filename"""
    return '_'.join(os.path.basename(filename).split('_')[:-1])

def group_files_by_category(data_dir):
    """Group image and mask files by their product category"""
    image_files = glob.glob(os.path.join(data_dir, "*.jpg"))
    mask_files = glob.glob(os.path.join(data_dir, "*_gt.png"))
    
    category_dict = defaultdict(lambda: {'images': [], 'masks': []})
    
    for img in image_files:
        category = extract_category_from_filename(img)
        category_dict[category]['images'].append(img)
    
    for mask in mask_files:
        base_name = '_'.join(os.path.basename(mask).split('_')[:-2])
        category = extract_category_from_filename(base_name + '.jpg')
        
        matching_images = [img for img in category_dict[category]['images'] 
                         if base_name in img]
        
        for img in matching_images:
            category_dict[category]['masks'].append((img, mask))
    
    return category_dict

def process_img_png_mask(img_path, mask_path, visualize=False):
    """Extract bounding box coordinates from mask"""
    try:
        img = Image.open(img_path)
        mask = Image.open(mask_path)
        mask_array = np.array(mask)
        
        if mask_array.size == 0:
            print(f"Warning: Empty mask in {mask_path}")
            return 0, 0, 0, 0
            
        rows = np.any(mask_array, axis=1)
        cols = np.any(mask_array, axis=0)
        
        ymin, ymax = np.where(rows)[0][[0, -1]]
        xmin, xmax = np.where(cols)[0][[0, -1]]
        
        if visualize:
            fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].imshow(img)
            ax[0].set_title("Original Image")
            ax[1].imshow(mask_array)
            rect = plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, 
                               linewidth=1, edgecolor='r', facecolor='none')
            ax[1].add_patch(rect)
            ax[1].set_title("Mask with Bounding Box")
            plt.show()
        
        return xmin, xmax, ymin, ymax
        
    except Exception as e:
        print(f"Error processing {img_path}: {str(e)}")
        return 0, 0, 0, 0

def mask_to_bbox(mask):
    """Improved mask to bbox conversion with better handling"""
    # Handle 3D mask by taking first channel if needed
    if len(mask.shape) == 3:
        mask = mask[0] if mask.shape[0] == 1 else mask
    
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    
    if not np.any(rows) or not np.any(cols):
        return [0, 0, 0, 0]  # Return empty bbox if no True values
    
    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    
    return [int(xmin), int(ymin), int(xmax-xmin), int(ymax-ymin)]

def evaluate_product_category(category_name, image_mask_pairs, max_display=10):
    """Fixed evaluation with proper COCO format and output handling"""
    print(f"\nEvaluating {category_name} (showing first {max_display} outputs)...")
    
    if not image_mask_pairs or len(image_mask_pairs) < 2:
        print("Error: Need at least 2 image-mask pairs for evaluation")
        return None
    
    # Initialize COCO ground truth with all required fields
    coco_gt = {
        "info": {
            "description": f"{category_name} product detection",
            "version": "1.0",
            "year": 2023,
            "contributor": "User",
            "date_created": "2023-01-01"
        },
        "licenses": [{
            "id": 1,
            "name": "Academic",
            "url": ""
        }],
        "images": [],
        "annotations": [],
        "categories": [{
            "id": 1,
            "name": "product",
            "supercategory": "object"
        }]
    }
    
    # Use first image-mask pair as reference
    first_img, first_mask = image_mask_pairs[0]
    print(f"Using reference image: {os.path.basename(first_img)}")
    print(f"Using reference mask: {os.path.basename(first_mask)}")
    
    xmin, xmax, ymin, ymax = process_img_png_mask(first_img, first_mask)
    print(f"Reference bounding box: x=[{xmin},{xmax}], y=[{ymin},{ymax}]")
    
    coco_dt = []
    
    for i, (img_path, mask_path) in enumerate(image_mask_pairs[1:]):
        if i >= max_display:  # Only show first N outputs
            clear_output(wait=True)
            print(f"Processing {len(image_mask_pairs)-1} images, showing first {max_display}...")
            break
            
        print(f"\nProcessing image {i+1}/{len(image_mask_pairs)-1}: {os.path.basename(img_path)}")
        
        try:
            # Process ground truth
            gt_mask = np.array(Image.open(mask_path))
            gt_bbox = mask_to_bbox(gt_mask)
            print(f"Ground truth bbox: {gt_bbox}")
            
            # Add to COCO ground truth
            image_id = i+1
            coco_gt["images"].append({
                "id": image_id,
                "width": gt_mask.shape[1],
                "height": gt_mask.shape[0],
                "file_name": img_path
            })
            
            rle = cocomask.encode(np.asfortranarray(gt_mask))
            rle['counts'] = rle['counts'].decode('ascii')
            
            coco_gt["annotations"].append({
                "id": image_id,
                "image_id": image_id,
                "category_id": 1,
                "bbox": gt_bbox,
                "area": int(gt_bbox[2] * gt_bbox[3]),
                "iscrowd": 0,
                "segmentation": rle
            })
            
            # Get SAM2 prediction
            video_segments = track_item_boxes(first_img, img_path, [([xmin, xmax, ymin, ymax], 1)], False)
            
            if not video_segments or 1 not in video_segments:
                print("Warning: No valid segments found in frame 1")
                continue
                
            # Get the prediction mask
            pred_mask = video_segments[1][1]  # Frame 1, Object ID 1
            
            # Convert mask to correct format if needed
            if len(pred_mask.shape) == 3:
                pred_mask = pred_mask[0] if pred_mask.shape[0] == 1 else pred_mask
            
            pred_bbox = mask_to_bbox(pred_mask)
            print(f"Predicted bbox: {pred_bbox}")
            
            # Skip if prediction is empty
            if pred_bbox == [0, 0, 0, 0]:
                print("Warning: Empty prediction - skipping")
                continue
                
            # Convert prediction to COCO format
            pred_rle = cocomask.encode(np.asfortranarray(pred_mask.astype(np.uint8)))
            pred_rle['counts'] = pred_rle['counts'].decode('ascii')
            
            coco_dt.append({
                "image_id": image_id,
                "category_id": 1,
                "bbox": pred_bbox,
                "score": 1.0,
                "segmentation": pred_rle
            })
            
            # Visual inspection for first few images
            if i < 3:
                plt.figure(figsize=(10,5))
                plt.subplot(1,2,1)
                plt.imshow(Image.open(img_path))
                plt.title("Original")
                plt.subplot(1,2,2)
                plt.imshow(pred_mask)
                plt.title("Prediction")
                plt.show()
            
        except Exception as e:
            print(f"Error processing {img_path}: {str(e)}")
            continue
    
    if not coco_dt:
        print("Error: No valid predictions generated!")
        return None
    
    try:
        # Create COCO objects
        coco_gt_obj = COCO()
        coco_gt_obj.dataset = coco_gt
        coco_gt_obj.createIndex()
        
        coco_dt_obj = coco_gt_obj.loadRes(coco_dt)
        
        # Initialize COCOeval
        coco_eval = COCOeval(coco_gt_obj, coco_dt_obj, 'bbox')
        
        # Evaluate and accumulate results
        coco_eval.evaluate()
        coco_eval.accumulate()
        
        # Capture the summary output
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        coco_eval.summarize()
        sys.stdout = old_stdout
        
        # Print the summary
        print(mystdout.getvalue())
        
        return coco_eval.stats
        
    except Exception as e:
        print(f"Error in COCO evaluation: {str(e)}")
        return None

def main():
    data_dir = r"C:\Users\dell\Desktop\Assignment\CMU10_3D\data_2D"
    
    # Group files by category
    category_dict = group_files_by_category(data_dir)
    
    # Evaluate each category
    results = {}
    for category, files in category_dict.items():
        # Create image-mask pairs
        image_mask_pairs = []
        for img in files['images']:
            base_name = os.path.splitext(os.path.basename(img))[0]
            masks = [mask for (img_path, mask) in files['masks'] 
                    if base_name in os.path.basename(mask)]
            
            if masks:
                image_mask_pairs.append((img, masks[0]))  # Using first mask
        
        if len(image_mask_pairs) < 2:
            print(f"Skipping {category} - needs at least 2 images with masks")
            continue
            
        # Save output to file
        output_file = f"{category}_results.txt"
        with open(output_file, 'w') as f:
            original_stdout = sys.stdout
            sys.stdout = f
            
            stats = evaluate_product_category(category, image_mask_pairs)
            
            sys.stdout = original_stdout
            print(f"Results saved to {output_file}")
        
        if stats is not None:
            results[category] = {
                "AP": stats[0],  # AP @ IoU=0.50:0.95
                "AP50": stats[1],  # AP @ IoU=0.50
                "AP75": stats[2],  # AP @ IoU=0.75
                "AR": stats[8]     # AR @ maxDets=100
            }
    
    # Print final results
    print("\n=== Final Results ===")
    if not results:
        print("No valid results generated - check individual category logs")
    else:
        for product, metrics in results.items():
            print(f"\n{product}:")
            print(f"  AP: {metrics['AP']:.3f}")
            print(f"  AP50: {metrics['AP50']:.3f}")
            print(f"  AP75: {metrics['AP75']:.3f}")
            print(f"  AR: {metrics['AR']:.3f}")

if __name__ == "__main__":
    main()

Results saved to rice_tuscan_results.txt

=== Final Results ===

can_chowder:
  AP: 0.198
  AP50: 0.257
  AP75: 0.257
  AR: 0.250

can_soymilk:
  AP: 0.087
  AP50: 0.109
  AP75: 0.109
  AR: 0.080

can_tomatosoup:
  AP: 0.128
  AP50: 0.142
  AP75: 0.142
  AR: 0.180

carton_oj:
  AP: 0.000
  AP50: 0.000
  AP75: 0.000
  AR: 0.000

carton_soymilk:
  AP: 0.289
  AP50: 0.693
  AP75: 0.022
  AR: 0.330

diet_coke:
  AP: 0.347
  AP50: 0.505
  AP75: 0.366
  AR: 0.380

hc_potroastsoup:
  AP: 0.298
  AP50: 0.590
  AP75: 0.230
  AR: 0.320

juicebox:
  AP: 0.136
  AP50: 0.234
  AP75: 0.083
  AR: 0.200

ricepilaf:
  AP: 0.044
  AP50: 0.054
  AP75: 0.054
  AR: 0.080

rice_tuscan:
  AP: 0.106
  AP50: 0.149
  AP75: 0.109
  AR: 0.110


In [14]:
import os
import glob
import shutil
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools import mask as cocomask
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from collections import defaultdict
import torch
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.build_sam import build_sam2_video_predictor
from IPython.display import clear_output
import sys
import warnings
from io import StringIO

# Suppress warnings and limit output
warnings.filterwarnings('ignore')

# SAM2 model setup
checkpoint = r"C:\Users\dell\Desktop\Assignment\sam2_hiera_tiny.pt"
model_cfg = r"C:\Users\dell\Desktop\Assignment\sam2\configs\sam2\sam2_hiera_t.yaml"

# Initialize SAM2 components
predictor_prompt = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint, device='cpu'))
sam2 = build_sam2(model_cfg, checkpoint, device='cpu', apply_postprocessing=False)
mask_generator = SAM2AutomaticMaskGenerator(sam2)
predictor_vid = build_sam2_video_predictor(model_cfg, checkpoint, device='cpu')

# Temporary directory for processing
tempfolder = "./tempdir"

def create_if_not_exists(dirname):
    if not os.path.exists(dirname):
        os.mkdir(dirname)

def cleardir(tempfolder):
    filepaths = glob.glob(tempfolder+"/*")
    for filepath in filepaths:
        os.unlink(filepath)

def show_mask(mask, ax, obj_id=None, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        cmap = plt.get_cmap("tab10")
        cmap_idx = 0 if obj_id is None else obj_id
        color = np.array([*cmap(cmap_idx)[:3], 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

TRACKING_THRESHOLD = 0.6  # Increased from 0.5
MIN_MASK_COVERAGE = 0.05  # Minimum % of mask coverage to consider valid
MAX_PADDING_RATIO = 0.2   # Increased from 0.1

def track_item_boxes(imgpath1, imgpath2, img1boxclasslist, visualize=True):
    """Enhanced tracking with dynamic padding and thresholding"""
    try:
        create_if_not_exists(tempfolder)
        cleardir(tempfolder)
        
        img1 = Image.open(imgpath1)
        img2 = Image.open(imgpath2)
        if img1.size != img2.size:
            img2 = img2.resize(img1.size)
        
        img1.save(os.path.join(tempfolder, "00000.jpg"))
        img2.save(os.path.join(tempfolder, "00001.jpg"))
        
        inference_state = predictor_vid.init_state(video_path="./tempdir")
        predictor_vid.reset_state(inference_state)
        
        # Dynamic padding based on object size
        for ([xmin, xmax, ymin, ymax], obj_id) in img1boxclasslist:
            box_width = xmax - xmin
            box_height = ymax - ymin
            pad_x = min(int(MAX_PADDING_RATIO * box_width), 100)  # Limit padding
            pad_y = min(int(MAX_PADDING_RATIO * box_height), 100)
            
            xmin = max(0, xmin - pad_x)
            xmax = min(img1.width, xmax + pad_x)
            ymin = max(0, ymin - pad_y)
            ymax = min(img1.height, ymax + pad_y)
            
            box = np.array([xmin, ymin, xmax, ymax], dtype=np.float32)
            _, out_obj_ids, _ = predictor_vid.add_new_points_or_box(
                inference_state=inference_state,
                frame_idx=0,
                obj_id=obj_id,
                box=box,
            )
        
        # Get predictions with adaptive threshold
        video_segments = {}
        for out_frame_idx, out_obj_ids, out_mask_logits in predictor_vid.propagate_in_video(inference_state):
            video_segments[out_frame_idx] = {
                obj_id: (out_mask_logits[i] > TRACKING_THRESHOLD).cpu().numpy().astype(bool)
                for i, obj_id in enumerate(out_obj_ids)
            }
        
        return video_segments
        
    except Exception as e:
        print(f"Tracking error: {str(e)}")
        return None

def extract_category_from_filename(filename):
    """Extract category name from filename"""
    return '_'.join(os.path.basename(filename).split('_')[:-1])

def group_files_by_category(data_dir):
    """Group image and mask files by their product category"""
    image_files = glob.glob(os.path.join(data_dir, "*.jpg"))
    mask_files = glob.glob(os.path.join(data_dir, "*_gt.png"))
    
    category_dict = defaultdict(lambda: {'images': [], 'masks': []})
    
    for img in image_files:
        category = extract_category_from_filename(img)
        category_dict[category]['images'].append(img)
    
    for mask in mask_files:
        base_name = '_'.join(os.path.basename(mask).split('_')[:-2])
        category = extract_category_from_filename(base_name + '.jpg')
        
        matching_images = [img for img in category_dict[category]['images'] 
                         if base_name in img]
        
        for img in matching_images:
            category_dict[category]['masks'].append((img, mask))
    
    return category_dict

def process_img_png_mask(img_path, mask_path, visualize=False):
    """Extract bounding box coordinates from mask"""
    try:
        img = Image.open(img_path)
        mask = Image.open(mask_path)
        mask_array = np.array(mask)
        
        if mask_array.size == 0:
            print(f"Warning: Empty mask in {mask_path}")
            return 0, 0, 0, 0
            
        rows = np.any(mask_array, axis=1)
        cols = np.any(mask_array, axis=0)
        
        ymin, ymax = np.where(rows)[0][[0, -1]]
        xmin, xmax = np.where(cols)[0][[0, -1]]
        
        if visualize:
            fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].imshow(img)
            ax[0].set_title("Original Image")
            ax[1].imshow(mask_array)
            rect = plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, 
                               linewidth=1, edgecolor='r', facecolor='none')
            ax[1].add_patch(rect)
            ax[1].set_title("Mask with Bounding Box")
            plt.show()
        
        return xmin, xmax, ymin, ymax
        
    except Exception as e:
        print(f"Error processing {img_path}: {str(e)}")
        return 0, 0, 0, 0

def mask_to_bbox(mask):
    """Improved mask to bbox conversion with better handling"""
    # Handle 3D mask by taking first channel if needed
    if len(mask.shape) == 3:
        mask = mask[0] if mask.shape[0] == 1 else mask
    
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    
    if not np.any(rows) or not np.any(cols):
        return [0, 0, 0, 0]  # Return empty bbox if no True values
    
    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    
    return [int(xmin), int(ymin), int(xmax-xmin), int(ymax-ymin)]

def evaluate_product_category(category_name, image_mask_pairs, max_display=10):
    """Fixed evaluation with proper COCO format and output handling"""
    print(f"\nEvaluating {category_name} (showing first {max_display} outputs)...")
    
    if not image_mask_pairs or len(image_mask_pairs) < 2:
        print("Error: Need at least 2 image-mask pairs for evaluation")
        return None
    
    # Initialize COCO ground truth with all required fields
    coco_gt = {
        "info": {
            "description": f"{category_name} product detection",
            "version": "1.0",
            "year": 2023,
            "contributor": "User",
            "date_created": "2023-01-01"
        },
        "licenses": [{
            "id": 1,
            "name": "Academic",
            "url": ""
        }],
        "images": [],
        "annotations": [],
        "categories": [{
            "id": 1,
            "name": "product",
            "supercategory": "object"
        }]
    }
    
    # Use first image-mask pair as reference
    first_img, first_mask = image_mask_pairs[0]
    print(f"Using reference image: {os.path.basename(first_img)}")
    print(f"Using reference mask: {os.path.basename(first_mask)}")
    
    xmin, xmax, ymin, ymax = process_img_png_mask(first_img, first_mask)
    print(f"Reference bounding box: x=[{xmin},{xmax}], y=[{ymin},{ymax}]")
    
    coco_dt = []
    
    for i, (img_path, mask_path) in enumerate(image_mask_pairs[1:]):
        if i >= max_display:  # Only show first N outputs
            clear_output(wait=True)
            print(f"Processing {len(image_mask_pairs)-1} images, showing first {max_display}...")
            break
            
        print(f"\nProcessing image {i+1}/{len(image_mask_pairs)-1}: {os.path.basename(img_path)}")
        
        try:
            # Process ground truth
            gt_mask = np.array(Image.open(mask_path))
            gt_bbox = mask_to_bbox(gt_mask)
            print(f"Ground truth bbox: {gt_bbox}")
            
            # Add to COCO ground truth
            image_id = i+1
            coco_gt["images"].append({
                "id": image_id,
                "width": gt_mask.shape[1],
                "height": gt_mask.shape[0],
                "file_name": img_path
            })
            
            rle = cocomask.encode(np.asfortranarray(gt_mask))
            rle['counts'] = rle['counts'].decode('ascii')
            
            coco_gt["annotations"].append({
                "id": image_id,
                "image_id": image_id,
                "category_id": 1,
                "bbox": gt_bbox,
                "area": int(gt_bbox[2] * gt_bbox[3]),
                "iscrowd": 0,
                "segmentation": rle
            })
            
            # Get SAM2 prediction
            best_pred = None
            for attempt in range(2):  # Try up to 2 different parameter sets
                video_segments = track_item_boxes(
                    first_img, img_path, 
                    [([xmin, xmax, ymin, ymax], 1)], 
                    visualize=(attempt==0 and i<3)
                )
                
                if not video_segments or 1 not in video_segments:
                    continue
                    
                pred_mask = video_segments[1][1]
                if len(pred_mask.shape) == 3:
                    pred_mask = pred_mask[0] if pred_mask.shape[0] == 1 else pred_mask
                
                coverage = np.mean(pred_mask)
                if coverage < MIN_MASK_COVERAGE:
                    print(f"Low coverage ({coverage:.2%}), retrying with adjusted parameters")
                    global TRACKING_THRESHOLD
                    TRACKING_THRESHOLD = max(0.3, TRACKING_THRESHOLD - 0.1)  # Lower threshold
                    continue
                
                best_pred = pred_mask
                break
            
            if best_pred is None:
                print("Warning: No valid prediction after retries")
                continue
                
            pred_bbox = mask_to_bbox(best_pred)
            coverage = np.mean(best_pred)
            print(f"Predicted bbox: {pred_bbox}, Coverage: {coverage:.1%}")
            
            if coverage < MIN_MASK_COVERAGE:
                print("Warning: Insufficient coverage - skipping")
                continue
            
            # Skip if prediction is empty
            if pred_bbox == [0, 0, 0, 0]:
                print("Warning: Empty prediction - skipping")
                continue
                
            # Convert prediction to COCO format
            pred_rle = cocomask.encode(np.asfortranarray(pred_mask.astype(np.uint8)))
            pred_rle['counts'] = pred_rle['counts'].decode('ascii')
            
            coco_dt.append({
                "image_id": image_id,
                "category_id": 1,
                "bbox": pred_bbox,
                "score": 1.0,
                "segmentation": pred_rle
            })
            
            # Visual inspection for first few images
            if i < 3:
                plt.figure(figsize=(10,5))
                plt.subplot(1,2,1)
                plt.imshow(Image.open(img_path))
                plt.title("Original")
                plt.subplot(1,2,2)
                plt.imshow(pred_mask)
                plt.title("Prediction")
                plt.show()
            
        except Exception as e:
            print(f"Error processing {img_path}: {str(e)}")
            continue
    
    if not coco_dt:
        print("Error: No valid predictions generated!")
        return None
    
    try:
        # Create COCO objects
        coco_gt_obj = COCO()
        coco_gt_obj.dataset = coco_gt
        coco_gt_obj.createIndex()
        
        coco_dt_obj = coco_gt_obj.loadRes(coco_dt)
        
        # Initialize COCOeval
        coco_eval = COCOeval(coco_gt_obj, coco_dt_obj, 'bbox')
        
        # Evaluate and accumulate results
        coco_eval.evaluate()
        coco_eval.accumulate()
        
        # Capture the summary output
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        coco_eval.summarize()
        sys.stdout = old_stdout
        
        # Print the summary
        print(mystdout.getvalue())
        
        return coco_eval.stats
        
    except Exception as e:
        print(f"Error in COCO evaluation: {str(e)}")
        return None

def main():
    data_dir = r"C:\Users\dell\Desktop\Assignment\CMU10_3D\data_2D"
    
    # Group files by category
    category_dict = group_files_by_category(data_dir)
    
    # Evaluate each category
    results = {}
    for category, files in category_dict.items():
        # Create image-mask pairs
        image_mask_pairs = []
        for img in files['images']:
            base_name = os.path.splitext(os.path.basename(img))[0]
            masks = [mask for (img_path, mask) in files['masks'] 
                    if base_name in os.path.basename(mask)]
            
            if masks:
                image_mask_pairs.append((img, masks[0]))  # Using first mask
        
        if len(image_mask_pairs) < 2:
            print(f"Skipping {category} - needs at least 2 images with masks")
            continue
            
        # Save output to file
        output_file = f"{category}_results.txt"
        with open(output_file, 'w') as f:
            original_stdout = sys.stdout
            sys.stdout = f
            
            stats = evaluate_product_category(category, image_mask_pairs)
            
            sys.stdout = original_stdout
            print(f"Results saved to {output_file}")
        
        if stats is not None:
            results[category] = {
                "AP": stats[0],  # AP @ IoU=0.50:0.95
                "AP50": stats[1],  # AP @ IoU=0.50
                "AP75": stats[2],  # AP @ IoU=0.75
                "AR": stats[8]     # AR @ maxDets=100
            }
    
    # Print final results
    print("\n=== Final Results ===")
    if not results:
        print("No valid results generated - check individual category logs")
    else:
        for product, metrics in results.items():
            print(f"\n{product}:")
            print(f"  AP: {metrics['AP']:.3f}")
            print(f"  AP50: {metrics['AP50']:.3f}")
            print(f"  AP75: {metrics['AP75']:.3f}")
            print(f"  AR: {metrics['AR']:.3f}")

if __name__ == "__main__":
    main()

Results saved to rice_tuscan_results.txt

=== Final Results ===

can_chowder:
  AP: 0.000
  AP50: 0.000
  AP75: 0.000
  AR: 0.000

carton_oj:
  AP: 0.028
  AP50: 0.069
  AP75: 0.036
  AR: 0.080

carton_soymilk:
  AP: 0.156
  AP50: 0.437
  AP75: 0.000
  AR: 0.180

diet_coke:
  AP: 0.178
  AP50: 0.208
  AP75: 0.208
  AR: 0.170

hc_potroastsoup:
  AP: 0.000
  AP50: 0.000
  AP75: 0.000
  AR: 0.000

ricepilaf:
  AP: 0.000
  AP50: 0.000
  AP75: 0.000
  AR: 0.000

rice_tuscan:
  AP: 0.098
  AP50: 0.109
  AP75: 0.109
  AR: 0.090
